# Import prometheus snapshot to another fabric slice

## Import

In [1]:
import os
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
try:
    fablib = fablib_manager()                
    #fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")
import mflib
print(f"MFLib version  {mflib.__version__} " )
from mflib.mflib import MFLib

MFLib version  1.0.0 


In [2]:
import importlib
import mf_data_transfer
importlib.reload(mf_data_transfer)
from mf_data_transfer import mf_data_import

## Slice Info

In [3]:
slice_name = "backup-testing"
node1_name = 'data-backup'

In [4]:
try:
    slice = fablib.get_slice(name=slice_name)
except Exception as e:
    print(f"Fail: {e}")
print (slice)

try:
    node = slice.get_node(name=node1_name)
except Exception as e:
    print(f"Fail: {e}")

-----------  ------------------------------------
Slice Name   backup-testing
Slice ID     a51c0f31-82a8-46b1-b19d-5e9216016dbf
Slice State  StableOK
Lease End    2023-06-13 00:02:12 +0000
-----------  ------------------------------------


## Untar the snapshot tar file

In [5]:
# Create the snapshot folder
prometheus_snapshot_path = "/home/ubuntu/prom_data"
node.execute(f"sudo mkdir -p {prometheus_snapshot_path}/snapshots")

 sudo: unable to resolve host data-backup: Name or service not known
 

('', 'sudo: unable to resolve host data-backup: Name or service not known\n')

In [6]:
# Untar the prometheus tar file, change the file name
prometheus_tar_file_name = "20230529T234802Z-4be04af16c86e4ea.tar"
prometheus_tar_path = "/home/ubuntu/prometheus/download"

cmd = f'sudo tar -xvf {prometheus_tar_path}/{prometheus_tar_file_name} -C {prometheus_snapshot_path}/snapshots'
node.execute(cmd, quiet=True)

('20230529T234802Z-4be04af16c86e4ea/\n20230529T234802Z-4be04af16c86e4ea/01H1MYJR5DEY9FCNRD15HZSVNN/\n20230529T234802Z-4be04af16c86e4ea/01H1MYJR5DEY9FCNRD15HZSVNN/tombstones\n20230529T234802Z-4be04af16c86e4ea/01H1MYJR5DEY9FCNRD15HZSVNN/index\n20230529T234802Z-4be04af16c86e4ea/01H1MYJR5DEY9FCNRD15HZSVNN/meta.json\n20230529T234802Z-4be04af16c86e4ea/01H1MYJR5DEY9FCNRD15HZSVNN/chunks/\n20230529T234802Z-4be04af16c86e4ea/01H1MYJR5DEY9FCNRD15HZSVNN/chunks/000001\n',
 'sudo: unable to resolve host data-backup: Name or service not known\n')

In [7]:
node.execute(f"sudo ls {prometheus_snapshot_path}/snapshots", quiet=True)

('20230529T234802Z-4be04af16c86e4ea\n',
 'sudo: unable to resolve host data-backup: Name or service not known\n')

## Start prometheus container

In [8]:
# Upload an example prometheus.yml file
node.upload_file(local_file_path="/home/fabric/work/data_transfer/prometheus_docker/prometheus.yml", 
                 remote_file_path='/home/ubuntu/prometheus.yml')

<SFTPAttributes: [ size=287 uid=1000 gid=1000 mode=0o100664 atime=1685406198 mtime=1685406198 ]>

In [9]:
# start the prometheus docker container
prom_snapshot_name= "20230529T234802Z-4be04af16c86e4ea"
prom_yml_path = "/home/ubuntu/prometheus.yml"
prometheus_snapshot_path = "/home/ubuntu/prom_data/snapshots"

cmd = f'''
          sudo docker run --rm -dit --name prometheus \
          --user 0:0 \
          -p 9090:9090 \
          -v {prometheus_snapshot_path}/{prom_snapshot_name}:/prometheus:rw \
          -v {prom_yml_path}:/etc/prometheus/prometheus.yml \
          prom/prometheus:v2.33.4 \
          --config.file=/etc/prometheus/prometheus.yml \
          --web.enable-admin-api \
          --web.enable-lifecycle \
          --storage.tsdb.path=/prometheus
      '''
node.execute(cmd)

f207407bdae221c54f5d9666276abe091d2e0b2b1868d83a315a76a1330a24a6
 sudo: unable to resolve host data-backup: Name or service not known
Unable to find image 'prom/prometheus:v2.33.4' locally
v2.33.4: Pulling from prom/prometheus
009932687766: Pulling fs layer
ff9264fbb6f4: Pulling fs layer
1ad6d9643fdd: Pulling fs layer
e6f7fea04459: Pulling fs layer
63fc05a36a59: Pulling fs layer
604ad6adddc4: Pulling fs layer
54b552d4bfbc: Pulling fs layer
a23b2328402c: Pulling fs layer
9ec38b0764ed: Pulling fs layer
cf8aa72409ac: Pulling fs layer
7104c0fa6750: Pulling fs layer
b782297611dd: Pulling fs layer
54b552d4bfbc: Waiting
a23b2328402c: Waiting
9ec38b0764ed: Waiting
cf8aa72409ac: Waiting
7104c0fa6750: Waiting
b782297611dd: Waiting
e6f7fea04459: Waiting
604ad6adddc4: Waiting
63fc05a36a59: Waiting
ff9264fbb6f4: Verifying Checksum
ff9264fbb6f4: Download complete
009932687766: Verifying Checksum
009932687766: Download complete
009932687766: Pull complete
ff9264fbb6f4: Pull complete
1ad6d9643fdd: Ver

('f207407bdae221c54f5d9666276abe091d2e0b2b1868d83a315a76a1330a24a6\n',
 "sudo: unable to resolve host data-backup: Name or service not known\nUnable to find image 'prom/prometheus:v2.33.4' locally\nv2.33.4: Pulling from prom/prometheus\n009932687766: Pulling fs layer\nff9264fbb6f4: Pulling fs layer\n1ad6d9643fdd: Pulling fs layer\ne6f7fea04459: Pulling fs layer\n63fc05a36a59: Pulling fs layer\n604ad6adddc4: Pulling fs layer\n54b552d4bfbc: Pulling fs layer\na23b2328402c: Pulling fs layer\n9ec38b0764ed: Pulling fs layer\ncf8aa72409ac: Pulling fs layer\n7104c0fa6750: Pulling fs layer\nb782297611dd: Pulling fs layer\n54b552d4bfbc: Waiting\na23b2328402c: Waiting\n9ec38b0764ed: Waiting\ncf8aa72409ac: Waiting\n7104c0fa6750: Waiting\nb782297611dd: Waiting\ne6f7fea04459: Waiting\n604ad6adddc4: Waiting\n63fc05a36a59: Waiting\nff9264fbb6f4: Verifying Checksum\nff9264fbb6f4: Download complete\n009932687766: Verifying Checksum\n009932687766: Download complete\n009932687766: Pull complete\nff9264fbb

In [10]:
node.execute("sudo docker ps")

CONTAINER ID   IMAGE                     COMMAND                  CREATED          STATUS          PORTS                                       NAMES
f207407bdae2   prom/prometheus:v2.33.4   "/bin/prometheus --c…"   17 seconds ago   Up 14 seconds   0.0.0.0:9090->9090/tcp, :::9090->9090/tcp   prometheus
43ca34ee7daf   data_transfer:1.0         "python3"                7 minutes ago    Up 7 minutes                                                data_transfer
 sudo: unable to resolve host data-backup: Name or service not known
 

('CONTAINER ID   IMAGE                     COMMAND                  CREATED          STATUS          PORTS                                       NAMES\nf207407bdae2   prom/prometheus:v2.33.4   "/bin/prometheus --c…"   17 seconds ago   Up 14 seconds   0.0.0.0:9090->9090/tcp, :::9090->9090/tcp   prometheus\n43ca34ee7daf   data_transfer:1.0         "python3"                7 minutes ago    Up 7 minutes                                                data_transfer\n',
 'sudo: unable to resolve host data-backup: Name or service not known\n')

## Query the metrics

In [11]:
node.execute("sudo curl http://localhost:9090/api/v1/label/__name__/values")

{"status":"success","data":["docker_containers","docker_probe_duration_seconds_bucket","docker_probe_duration_seconds_count","docker_probe_duration_seconds_sum","docker_probe_list_containers_duration_seconds_bucket","docker_probe_list_containers_duration_seconds_count","docker_probe_list_containers_duration_seconds_sum","docker_probe_list_containers_failed_total","dotnet_collection_count_total","dotnet_total_memory_bytes","go_gc_cycles_automatic_gc_cycles_total","go_gc_cycles_forced_gc_cycles_total","go_gc_cycles_total_gc_cycles_total","go_gc_duration_seconds","go_gc_duration_seconds_count","go_gc_duration_seconds_sum","go_gc_heap_allocs_by_size_bytes_total_bucket","go_gc_heap_allocs_by_size_bytes_total_count","go_gc_heap_allocs_by_size_bytes_total_sum","go_gc_heap_allocs_bytes_total","go_gc_heap_allocs_objects_total","go_gc_heap_frees_by_size_bytes_total_bucket","go_gc_heap_frees_by_size_bytes_total_count","go_gc_heap_frees_by_size_bytes_total_sum","go_gc_heap_frees_bytes_total","go_g

('{"status":"success","data":["docker_containers","docker_probe_duration_seconds_bucket","docker_probe_duration_seconds_count","docker_probe_duration_seconds_sum","docker_probe_list_containers_duration_seconds_bucket","docker_probe_list_containers_duration_seconds_count","docker_probe_list_containers_duration_seconds_sum","docker_probe_list_containers_failed_total","dotnet_collection_count_total","dotnet_total_memory_bytes","go_gc_cycles_automatic_gc_cycles_total","go_gc_cycles_forced_gc_cycles_total","go_gc_cycles_total_gc_cycles_total","go_gc_duration_seconds","go_gc_duration_seconds_count","go_gc_duration_seconds_sum","go_gc_heap_allocs_by_size_bytes_total_bucket","go_gc_heap_allocs_by_size_bytes_total_count","go_gc_heap_allocs_by_size_bytes_total_sum","go_gc_heap_allocs_bytes_total","go_gc_heap_allocs_objects_total","go_gc_heap_frees_by_size_bytes_total_bucket","go_gc_heap_frees_by_size_bytes_total_count","go_gc_heap_frees_by_size_bytes_total_sum","go_gc_heap_frees_bytes_total","go

In [12]:
metric_name = "node_network_receive_packets_total"
time_range="3h"
node.execute(f"sudo curl -G 'http://localhost:9090/api/v1/query?query={metric_name}[{time_range}]'", quiet=True)

('{"status":"success","data":{"resultType":"matrix","result":[{"metric":{"__name__":"node_network_receive_packets_total","device":"docker0","instance":"Node1","job":"node","node_name":"Node1"},"values":[[1685402740.480,"0"],[1685402770.480,"0"],[1685402800.480,"0"],[1685402830.483,"0"],[1685402860.480,"0"],[1685402890.480,"0"],[1685402920.480,"0"],[1685402950.480,"0"],[1685402980.480,"0"],[1685403010.480,"0"],[1685403040.480,"0"],[1685403070.480,"0"],[1685403100.480,"0"],[1685403130.480,"0"],[1685403160.480,"0"],[1685403190.480,"0"],[1685403220.480,"0"],[1685403250.480,"0"],[1685403280.480,"0"],[1685403310.480,"0"],[1685403340.480,"0"],[1685403370.480,"0"],[1685403400.480,"0"],[1685403430.480,"0"],[1685403460.480,"0"],[1685403490.480,"0"],[1685403520.480,"0"],[1685403550.480,"0"],[1685403580.480,"0"],[1685403610.480,"0"],[1685403640.480,"0"],[1685403670.480,"0"],[1685403700.480,"0"],[1685403730.480,"0"],[1685403760.480,"0"],[1685403790.480,"0"],[1685403820.480,"0"],[1685403850.480,"0"]

In [13]:
metric_name = "node_network_receive_packets_total"
start = "2023-05-29T23:48:00.000Z"
end = "2023-05-30T01:00:00.000Z"
cmd = f'''
       sudo curl -G "http://localhost:9090/api/v1/query_range?query={metric_name}&start={start}&end={end}&step=15s"
       '''
node.execute(cmd, quiet=True)

('{"status":"success","data":{"resultType":"matrix","result":[{"metric":{"__name__":"node_network_receive_packets_total","device":"docker0","instance":"Node1","job":"node","node_name":"Node1"},"values":[[1685404080,"0"],[1685404095,"0"],[1685404110,"0"],[1685404125,"0"],[1685404140,"0"],[1685404155,"0"],[1685404170,"0"],[1685404185,"0"],[1685404200,"0"],[1685404215,"0"],[1685404230,"0"],[1685404245,"0"],[1685404260,"0"],[1685404275,"0"],[1685404290,"0"],[1685404305,"0"],[1685404320,"0"],[1685404335,"0"],[1685404350,"0"]]},{"metric":{"__name__":"node_network_receive_packets_total","device":"docker0","instance":"Node2","job":"node","node_name":"Node2"},"values":[[1685404080,"0"],[1685404095,"0"],[1685404110,"0"],[1685404125,"0"],[1685404140,"0"],[1685404155,"0"],[1685404170,"0"],[1685404185,"0"],[1685404200,"0"],[1685404215,"0"],[1685404230,"0"],[1685404245,"0"],[1685404260,"0"],[1685404275,"0"],[1685404290,"0"],[1685404305,"0"],[1685404320,"0"],[1685404335,"0"],[1685404350,"0"],[1685404

## View data in the web ui

##### RUN 'ssh -L 10030:localhost:9090 -F ssh_config -i slice_key ubuntu@node_ip'
##### Then go to http://localhost:10030